Questa sezione tratta la correlazione delle statistiche. 

Il calcio moderno è diventato molto caro, sopratutto per il reparto di attacco. Le società chiedono tantissimi soldi per un giocatore "solamente" perchè ha realizzato molti Gol o Assist. Non tutte le squadre, però, hanno una disponibilità economica elevata per puntare su un attaccante già affermato che ha realizzato numeri altissimi. L'idea dietro a questa analisi è quella di capire se ci sono delle statistiche ben relazionate con quelle sopracitate, che una squadra può andare ad analizzare per trovare un attaccante d'impatto per la propria squadra. Vogliamo trovare quali sono queste statisitche.

In [1]:
# Importo le librerie necessarie per il codice

import pandas as pd
import plotly.express as px
pd.set_option('display.max_columns', None)

# Elenco dei file da leggere
files = ['attaccanti_23.xlsx', 'attaccanti_22.xlsx', 'attaccanti_21.xlsx', 'attaccanti_20.xlsx', 'attaccanti_19.xlsx']

# Faccio un dizionario per mappare i file agli anni
year_mapping = {'attaccanti_23.xlsx': 2023,'attaccanti_22.xlsx': 2022,'attaccanti_21.xlsx': 2021,'attaccanti_20.xlsx': 2020,'attaccanti_19.xlsx': 2019}

In [2]:
# Questa sezione è divisa in due parti. 

# La prima parte si occupa esclusivamente  delle tabelle di correlazione infatti, una volta scaricati i file,
# si potranno visualizzare cinque quadrati interattivi che riportano come si correlano tra di loro le statistiche 
# selezionate nel dataframe. 
# Alla fine per comodità verranno riportate le colonne usate.

# Nella seconda parte abbiamo inserito una funzione che ha come scopo quello di scoprire quali sono le 5 statistiche meglio e peggio
# correlate con quella desiderata data in input. Ed una funzione che restituisce l'elenco di tutti gli indici di correlazione
# con le altre statistiche. Ed infine una terza funzione che ha come scopo quello di vedere come sono correlate
# due statistiche specifiche, date in input alla funzione, e vedere come la loro correlazione è cambiata negli ultimi 5 anni,
# sicuramente utile per scoprire se la correlazione o scorrelazione di due dati sia casuale o una cosa costante nel tempo.

Prima di tutto bisogna fare un processo di 'data cleaning' ovvero una pulizia del dataframe, aggiungendo colonne che possono essere utili oppure togliendone altre che non voglio prendere in considerazione.

Per prima cosa togliamo i dati dei cartellini gialli e rossi collezionati dai giocatori, in quanto rappresentano la disciplina di questi e non le loro capacità balistiche. Dopodiche eliminiamo le colonne relative ai calci piazzati in quanto rigori e punizioni hanno i propri specialisti scelti dall'allenatore, che non devono essere obbligatoriamente attaccanti, che battono tutti quelli che vengono assegnati nelle partite. Non sarebbe quindi equo considerare una statistica alla quale la maggior parte dei giocatori non prendono neanche parte.

Ovviamente scartiamo le celle relative al Nome del giocatore, la Squadra per cui gioca e la sua Nazionalità.
Subito dopo togliamo quelle colonne che rappresentano "operazioni" fra altre statistiche, come ad esempio somme fra due dati, percentuali o rapporti, in quanto non sono statistiche di gioco (realizzate sul campo dal giocatore) ma create ad hoc per altri tipi di analisi non inerenti al nostro.

In [2]:
# Questa parte di codice crea 5 figure che rappresentano il riquadro di correlazione fra le statistiche per annata

correlations = []
years = []

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad', 'G+A', 'G-PK','SoT%','PK',
       'PKatt',  'FK', 'G/Sh', 'G/SoT', 'G-xG','CrdY', 'CrdR', '2CrdY', 'Cmp%','Succ%','Won%']

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] # Creo la colonna dei duelli aerei, può essere utile per l'analisi

    # Uso .drop per rimuovere dal data frame quelle statistiche che non voglio usare in questa analisi
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()  # Calcolo la matrice di correlazione

    year = year_mapping[file]  # Ottengo l'anno dal file usando il mapping
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    # Ottengo in output il grafico di correlazione

    fig.show()

# Inserisco questa riga di codice in modo da poter visualizzare le colonne usate per l'analisi
print(f'Le colonne usate per questa analisi sono {list(data_frame.columns)}')


Le colonne usate per questa analisi sono ['Age', 'MP', 'Starts', 'Min', 'Gls', 'Ast', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'Dist', 'Cmp', 'Att', 'xA', 'KP', 'PPA', 'CrsPA', 'SCA', 'GCA', 'Tkl', 'TklW', 'Mid 3rd', 'Att 3rd', 'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Carries', 'TotDist', 'Fld', 'Off', 'Crs', 'Won', 'Lost', 'Duels']


In [2]:
# Creo una funzione che mi permette di scoprire quali sono le statistiche meglio e peggio correlate con una desiderata

def best_worst(stat):
 for file in files:
    year = year_mapping[file]
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    if stat in corr_matrix.columns:
        # Ordina le correlazioni con la statistica scelta
        sorted_corr = corr_matrix[stat].sort_values(ascending=False)

        # Trova le 5 migliori e peggiori correlazioni (cambiare i range per saperne di più o di meno)
        best_corr = sorted_corr[1:6]  # Parto da 1 per evitare la correlazione della statistica con se stessa
        worst_corr = sorted_corr[-5:]

        print(f"Anno {year}: Le 5 statistiche meglio correlate con {stat} sono:\n{best_corr}")
        print(f"Anno {year}: Le 5 statistiche peggio correlate con {stat} sono:\n{worst_corr}")
    else:
        print(f"La statistica {stat} non è presente nel dataset. Controlla l'input")

# Creo una funzione che restituisce l'elenco degli indci di correlazione di una statistica con le altre anno per anno

def correlazioni_annuali(stat):
    for file in files:
        year = year_mapping[file]
        data_frame = pd.read_excel(file)  
        data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

        data_frame.drop(columns=colonne_da_eliminare, inplace=True)

        corr_matrix = data_frame.corr()

        if stat in corr_matrix.columns:
            sorted_corr = corr_matrix[stat].sort_values(ascending=False)

            print(f"Anno {year}: Le correlazioni per {stat} sono:\n{sorted_corr}\n")
        else:
            print(f"La statistica {stat} non è presente nel dataset dell'anno {year}.")
        
# Creo una seconda funzione che mi dice l'indice di correlazione di due statistiche specifiche
# ed il cambiamento di questo nel corso delle stagioni

def correlazione(stat1,stat2):
 correlations = []

 for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost']

    data_frame.drop(columns=colonne_da_eliminare, inplace=True)

    corr_matrix = data_frame.corr()

    year = year_mapping[file]

    # Verifica se entrambe le statistiche esistono nella matrice di correlazione
    if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
        correlation_value = corr_matrix.loc[stat1, stat2]
        correlations.append((year, correlation_value))
        #print(f"Anno {year}: Correlazione tra {stat1} e {stat2} = {correlation_value}")
    else:
        correlations.append((year, None))

# Creazione del DataFrame per il grafico
 if stat1 in corr_matrix.columns and stat2 in corr_matrix.columns:
    correlation_df = pd.DataFrame(correlations, columns=['Anno', 'Correlazione'])
    
    # Crea il grafico dell'andamento della correlazione nel tempo
    fig = px.line(correlation_df, x='Anno', y='Correlazione', title=f"Correlazione tra {stat1} e {stat2}", markers=True)
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(range = [-1,1], dtick = 0.10)
    fig.show()
 else:
    print("Nessun grafico disponibile: non sono state calcolate correlazioni valide. Controlla l'input")

Otteniamo in output i 5 grafici richiesti relativi alle 5 stagioni prese in considerazione. La barra sulla destra indica l'indice di correlazione in base al colore del quadrato relativo a due colonne. Muovendo il cursore sulle colonne si può vedere le due statistiche prese in considerazione ed il loro indice di correlazione. 

La presenza dei colori ci aiuta subito a notare che i grafici si assomigliano molto, nonostante vengano presi in considerazione 5 anni diversi nei quali hanno giocato molti giocatori diversi stagione per stagione. 
Immaginiamo quindi che gli indici di correlazione rimangano abbastanza costanti durante gli anni. Questo ci fa molto comodo, in quanto ci rassicura sul fatto che i risultati che troveremo non saranno casuali ma validi nel corso degli anni.

Oltre a questa osservazione possiamo anche notare, riconoscendo i colori, che ci sono statistiche mal correlate con tutte le altre. Salta subito all'occhio la colonna relativa ai Minuti giocati, indicata con 'Min'.

In [ ]:
stat = 'Min'
best_worst(stat)

stat1 = 'Min'
stat2 = 'Gls'
correlazione(stat1,stat2)

stat2 = 'Ast'
correlazione(stat1,stat2)

Non c'è bisogno di richiamare tutti i valori, ci basta usare le funzioni per confermare ciò che pensavamo. 
Nel corso degli anni il range di correlazione risulta sempre essere vicino allo 0. Questo è sicuramente il primo risultato grosso ed inaspettato di questa analisi in quanto ci aspettavamo che un minutaggio elevato avesse un grosso impatto sulle statistiche del singolo. 
Vediamo come l'andamento dell'indice di correlazione risulta costantemente vicino allo 0 nel corso degli anni.

Da un certo punto di vista però è un risultato che ci piace in quanto suggerisce che il rendimento di un attaccante non dipenda banalmente da quanto gioca ma da come gioca. Tutto ciò da ancora più valore alla nostra ricerca.

Questo risultato interessante appena trovato sui minuti ci suggerisce di testare le correlazioni delle altre statistiche non di campo ovvero 'Age' 'Starts' 'MP', rispettivamente, Età del giocatore, numero di partite titolare e numero totale di partite in cui ha giocato. Analizziamole separatamente una per volta.

In [ ]:
stat = 'Age'
correlazioni_annuali(stat)

stat1 = 'Age'
stat2 = 'Gls'
correlazione(stat1,stat2)

stat2 = 'Ast'
correlazione(stat1,stat2)

Nel 2023 il range di correlazione (esclusa ovviamente se stessa) è [0.276579, -0.098984], nel 2022 [0.308650, -0.070421], nel 2021 [0.320632, -0.119382], nel 2020 [0.352202, -0.090711] e nel 2019 [0.404946, -0.017619]. Possiamo notare come nel corso degli anni gli indici di correlazione siano rimasti in un range simile, vicino allo 0. Altro risultato inaspettato, in quanto pensavamo che l'esperienza di un giocatore più anziano oppure la freschezza e fantasia di un giovane potessero influire sulla performance di questo. Le nostre statistiche di interesse, ovviamente, stanno nei range appena citati e quindi vale lo stesso discorso.

In [ ]:
stat = 'Starts'
correlazioni_annuali(stat)

stat1 = 'Starts'
stat2 = 'Gls'
correlazione(stat1,stat2)

stat2 = 'Ast'
correlazione(stat1,stat2)

Il discorso qui è molto diverso, i range di correlazione sono molto più ampi e raggiungono numeri più alti. Nel 2023 [0.931046, -0.352069], nel 2022 [0.917525, -0.351932], nel 2021 [0.930689, -0.265947], nel 2020 [0.923731, -0.328074] e nel 2019 [0.916789, -0.379672]. Questa statistica è quindi correlata bene con alcune e male con altre. Notiamo, inoltre, come si relazioni abbastanza bene con le statistiche interessanti. Il risultato in realtà è abbastanza prevedibile in quanto è nell'interesse delle squadre schierare titolari i giocatori migliori che quindi registrano dati migliori.

In [ ]:
stat = 'MP'
correlazioni_annuali(stat)

stat1 = 'MP'
stat2 = 'Gls'
correlazione(stat1,stat2)

stat2 = 'Ast'
correlazione(stat1,stat2)

Il discorso riguardante il numero di partite giocate è il medesimo a quello precedente, è nell'interesse della squadra far giocare i giocatori migliori e quindi ci aspettavamo che fosse ben correlato.

Adesso ci concentriamo su quei dati che un attaccante registra quando messo a confronto con gli avversari, ovvero dribbling, contrasti, numero di tocchi, conduzioni di palla, distanza percorsa palla al piede, falli subiti e duelli aerei. Fra queste sono presenti colonne chiamate Mid 3rd e Att 3rd, questo indica in che area di campo viene registrata la statistica.

In [ ]:
correlations = []
years = []

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'xG', 'PrgP', 'PrgR', 'Sh', 'SoT', 'SoT%',
       'G/Sh', 'G/SoT', 'Dist', 'FK', 'G-xG', 'Cmp', 'Att', 'Cmp%', 'xA', 'KP',
       'PPA', 'CrsPA', 'SCA', 'Succ%','2CrdY', 'Off', 'Crs','Won%']

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] 
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    year = year_mapping[file] 
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    fig.show()

Dal range di correlazione indicato sulla destra capiamo come alcune statistiche siano correlate male e dal colore possiamo individuarle. Le colonne relative ai duelli aerei che sono correlate decentemente solamente con i Gol, quelle relative ai contrasti hanno indici bassi con quelle che ci interessano.
Ci aspettavamo che i falli subiti fossero ben correlati, in quanto una squadra potrebbe ricorrere ai falli per fermare un giocatore forte ed invece gli indici di correlazione sono abbastanza normali, si aggirano intorno al 0.6.
Sono relazionati molto bene invece i dati sui dribbling, con i dati di Assist, inoltre vediamo quelle di conduzioni di palla e in aggiunta quelle relative alla distanza percorsa palla al piede. 

In [ ]:
#Correlazione delle statistiche sopracitate nel tempo
# Assist

stat1 = 'Ast'
stat2 = 'Carries'
correlazione(stat1,stat2)

stat2 = 'TotDist'
correlazione(stat1,stat2)

stat2 = 'Att.1'
correlazione(stat1,stat2)

stat2 = 'Succ'
correlazione(stat1,stat2)

# Gol

stat1 = 'Gls'
stat2 = 'Carries'
correlazione(stat1,stat2)

stat2 = 'TotDist'
correlazione(stat1,stat2)

stat2 = 'Att.1'
correlazione(stat1,stat2)

stat2 = 'Succ'
correlazione(stat1,stat2)


Questo risultato ci dice che un giocatore fisico che vince molti contrasti e duelli aerei non sia del tutto necessario al fine dei Gol e gli altri dati di riferimento, piuttosto dovrebbe concentrarsi nell'ingaggiare un attaccante più tecnico ed agile che sia in grado di saltare l'uomo e portare palla. Le due caratteristiche non si escludono a vicenda, un giocatore può essere molto tecnico e anche fisico, stiamo dando una sorta di precedenza quando si va a studiare le doti principali di un atleta.

Analizziamo finalmente le statistiche di gioco, ovvero quelle relative ai tiri, passaggi, i calci piazzati. Ma prima occupiamoci del concetto di Gol ed Assist attesi. Questi dati indicano la probabilità di ogni tiro di essere gol o di un passaggio di essere un assist. 

In [7]:
colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'SoT%',
       'G/Sh', 'G/SoT', 'FK', 'G-xG','Cmp%','Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld', 'Duels', 'Won', 'Lost',
       'Won%']

In [ ]:
stat1 = 'Gls'
stat2 = 'xG'
correlazione(stat1,stat2)

stat1 = 'Ast'
stat2 = 'xA'
correlazione(stat1,stat2)

Il dato sui gol attesi tiene in considerazione tantissime variabili, ad esempio della difficoltà oggettiva del tiro, del numero di difensori presenti a contrastare o disturbare il tiro dell'attaccante, della qualità di questi difensori, tiene conto della posizione da cui sta tirando questo giocatore ed i risultati dei tiri fatti da altri giocatori in quella posizione, tiene conto del piede con cui calcia il giocatore, della sua posizione del corpo al momento dell'impatto e moltissime altre variabili. Tutti questi dati restituiscono la percentuale di un giocatore di segnare con il tiro che ha fatto, ad esempio un tiro da centrocampo avrà un indice di xG molto minore rispetto ad un calcio di rigore, oppure se due calciatori fanno due tiri identici il calciatore migliore avrà un indice di xG più alto. Dalla spiegazione riusciamo a comprendere come mai i due dati siano correlati quasi alla perfezione. Il dato sugli xG è frutto di un calcolo realizzato da intelligenze artificiali di ultimissima generazione che quindi riescono a fare una stima molto precisa.

Il concetto di Assist atteso è il medesimo, indica la probabilità che ogni passaggio che porta al tiro diventi un assist. Ed anche questo tiene conto di tantissime variabili. Il dato di correlazione è leggermente inferiore negli anni, perchè il dato non tiene in considerazione del giocatore che riceve la palla, infatti calcola solamente la probabilità che un determinato passaggio diventi assist. Logicamente un giocatore forte riuscirà a concretizzare il passaggio più spesso rendendo la differenza fra assist ed assist attesi del compagno minore. 

In [ ]:
correlations = []
years = []

colonne_da_eliminare = ['Player', 'Nation', 'Pos', 'Squad','Age', 'MP', 'Starts', 'Min', 'G+A', 'G-PK', 'PK',
       'PKatt', 'CrdY', 'CrdR', 'SoT%', 'xG', 'xA',
       'G/Sh', 'G/SoT', 'FK', 'G-xG','Cmp%','Tkl', 'TklW', 'Mid 3rd', 'Att 3rd',
       'Touches', 'Mid 3rd.1', 'Att 3rd.1', 'Att.1', 'Succ', 'Succ%',
       'Carries', 'TotDist', '2CrdY', 'Fld', 'Duels', 'Won', 'Lost',
       'Won%']

for file in files:
    
    data_frame = pd.read_excel(file)
    data_frame['Duels'] = data_frame['Won'] + data_frame['Lost'] 
    data_frame.drop(columns=colonne_da_eliminare, inplace=True) 

    corr_matrix = data_frame.corr()

    year = year_mapping[file] 
    years.append(year)

    fig = px.imshow(corr_matrix, title=f"Matrice di Correlazione - Anno {year}")
    fig.show()

In questi grafici notiamo come la statistica sulla distanza media di tiro abbia un indice di correlazione vicino allo zero e come le statistiche di passaggio siano ben relazionate tra di loro.
 
Il primo risultato importante, relativamente logico, è dato dall'ottima correlazione dalle statistiche sui passaggi e gli assist, spiccano i dati di passaggi prograssivi, i passaggi chiave e passaggi in area di rigore (PrgP, KP e PPA). Questo perchè un giocatore che passa meglio il pallone e fa passaggi più pericolosi realizzerà più assist.

Per quanto riguarda i gol oltre alle logiche correlazioni con i dati di tiri e tiri in porta troviamo anche un dato interessante che è la correlazione tra i passaggi progressivi ricevuti e i gol realizzati. Questo ci suggerisce che un attaccante che sa fare dei buoni movimenti senza palla e riesce a dettare il passaggio al compagno tendenzialmente fa più gol. Risultato interessante infatti questo vuol dire che un modo per aumentare il numero di gol realizzati da un attaccante può essere farlo lavorare sui movimenti e gli inserimenti. 

Vediamo infine come i dati sui tiri hanno un indice di circa 0.65 nel corso degli anni, simile a quello dei passaggi con i gol. Ovviamente esistono degli attaccanti completi che sanno sia passare bene la palla ai compagni che fare gol, ma sicuramente se una squadra ha più bisogno di un attaccante che faccia tanti gol rispetto a tanti assist dovrà concentrarsi su un calciatore che tira molte volte piuttosto che uno che sa passare bene la palla, viceversa se ha bisogno di assist dovrà ricercare un buon passatore piuttosto che un tiratore.

Le statistiche di chance da gol e da tiro create sono correlate bene con entrambe.


In [ ]:
# Assist

stat1 = 'Ast'
stat2 = 'KP'
correlazione(stat1,stat2)

stat2 = 'PPA'
correlazione(stat1,stat2)

stat2 = 'PrgP'
correlazione(stat1,stat2)

stat2 = 'GCA'
correlazione(stat1,stat2)

stat2 = 'SCA'
correlazione(stat1,stat2)

# Gol

stat1 = 'Gls'
stat2 = 'Sh'
correlazione(stat1,stat2)

stat2 = 'SoT'
correlazione(stat1,stat2)

stat2 = 'PrgR'
correlazione(stat1,stat2)

stat2 = 'GCA'
correlazione(stat1,stat2)

stat2 = 'SCA'
correlazione(stat1,stat2)

CONCLUSIONI

L'analisi sulla correlazione delle statistiche degli attaccanti conferma la nostra tesi, in quanto vediamo come esistono effettivamente delle statistiche ben correlate con i gol, gli assist e le chance da gol create. Abbiamo visto ed argomentato i vari risultati giustificando il perchè degli indici di correlazione e in conclusione siamo riusciti a stabilire una collezione più ristretta di statistiche che una squadra può andare a studiare per trovare un attaccante che possa migliorare il proprio reparto d'attacco e per le quali invece non valga la pena perdere tempo.

Per riassumere l'analisi possiamo quindi dire che una squadra non deve affidarsi all'età dell'atleta perchè non ha un indice di correlazione altissimo, inoltre abbiamo scoperto come la fisicità non è tutto ma bisogna dare la priorità all'agilità e ai fondamentali di dribbling e controllo palla. Una squadra deve sicuramente affidarsi alle tecnologie che indicano gli xG e gli xA in quanto abbiamo notato essere molto coerenti poi con il numero di gol ed assist effettivi. Ed infine possiamo dire che le statistiche da controllare immediatamente sono quelle relative ai tiri del giocatore, le chance da tiro e da gol create e la sua capacità di fare movimenti senza palla. Infine anche la sua dote di passaggio e la sua visione di gioco.